In [ ]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

#database setup
engine = create_engine("sqlite:///./Resources/hawaii.sqlite")

Base = automap_base()
Base.prepare(engine, reflect = True)

#save regerence to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

#create our session link from python to DB
session = Session(engine)

#flask set up 
app = Flask(__name__)

#Flask routes endpoint
@app.route("/")
def welcome():
    return(
        f" Welcome to the Hawaii Climate Analaysis API! <br/>"
        f" Available Routes <br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/temp/start/end")

@app.route("/api/v1.0/precipitation")
def precipitation():
    # return precipitation data from last year
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    
    #query for the date and precipitation for the last year 
    precipitation = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= prev_year).all()
    
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)

@app.route("/api/v1.0/stations")
def stations():
    results= session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations)

@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days = 365)
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= prev_year).all()
    
    temps = list(np.ravel(results))
    
    return jsonify(temps)

@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")
def stats(start = None, end = None):
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    
    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).all()
        
        temps = list(np.ravel(results))
        return jsonify(temps)
    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps)
    

if __name__== '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Feb/2020 23:28:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:28:26] "GET /api/v1.0/temp/start/end HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:28:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2020 23:28:43] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:28:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:28:52] "GET //api/v1.0/stations HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2020 23:28:54] "GET /api/v1.0/stations HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:28:54] "GET /api/v1.0/stations HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:29:12] "GET /api/v1.0/tobs HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:29:22] "GET /api/v1.0/temp/start/end HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:30:23] "GET /api/v1.0/temp/2016-08-23/2016-08-30 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2020 23:30:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2020 23: